# Bag of Words Sentiment Analyse von McDonalds Reviews

## Zielsetzung:
Die Reviews haben die Labels '1 star', '2 stars', '3 stars', '4 stars' und '5 stars'. 
Das reine Raten, wie der Reviewer bewertet hat, würde eine 20% Prozentige Trefferwahrscheinlichkeit haben. Dieses Benchmarkt muss unsere Analyse mindestens knacken um Sinnvoll zu sein. 

Alle Imports:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import TextVectorization
from tensorflow import keras
from keras import layers
import seaborn as sns
sns.set()

# use graphics in retina format
%config InlineBackend.figure_format = 'retina'

2023-07-03 16:27:17.729118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Datenvorbereitung inkl. Behandlung bekannter Probleme

### Laden der Daten

In [ ]:
import os
if not os.path.exists('mcdonalds-store-reviews.zip'):
    print("Downloading dataset...")
    !kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
if os.path.exists('mcdonalds-store-reviews.zip'):
    print("Unziping dataset...")
    !unzip -n mcdonalds-store-reviews.zip

In [2]:
df = pd.read_csv('McDonald_s_Reviews.csv', encoding="latin-1")
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


### Encoding der Daten


#### Umgang mit Strings als Label
Star Ratings werden gerade noch als 1 Star String bis zu einem 5 star String gespeichert. Mit einem numerischen System können neuronale Netzwerke besser umgehen, deshalb formatiere ich die labels zu interges zwischen 1 und 5.

In [3]:
train_data = df['review'].to_numpy()
label_data_strings = df['rating']
label_data = np.array([int(item.split()[0]) for item in label_data_strings])

Um die Verteilung der Ratings besser zu verstehen koennen wir die Daten visualieren.

In [ ]:
sns.displot(label_data)

### Aufteilung in Trainings und Validierungsdaten 

In [4]:
train_data, test_data, train_label, test_label = train_test_split(train_data, label_data, test_size=0.2, random_state=42)

print(train_data.shape)
print(test_data.shape)
print(train_label.shape)
print(test_label.shape)

### Text zu Dictonary Repräsentationen umbauen
Als Best Practice für Word Classification gibt das Buch Deep Learning with Python die Nutzung von 20000 Token an.
Als Vectorication Mehotik. N-Gramme sind Sequenzen von aufeinanderfolgenden Elementen in einer gegebenen Textsequenz. 
Ein N-Gramm mit einem Wert von N besteht aus N aufeinanderfolgenden Elementen, die in der Regel Wörter oder Zeichen sind. Zum Beispiel besteht ein 2-Gramm (auch Bigramm genannt) aus zwei aufeinanderfolgenden Wörtern in einem Text, während ein 3-Gramm (Trigramm) aus drei aufeinanderfolgenden Wörtern besteht.

Außerdem nutzen wir eine Multi-Hot-Kodierung als Ausgabe.
Bei der Multi-Hot-Kodierung wird für jedes Merkmal eine binäre Darstellung verwendet, bei der jedes Merkmal durch eine Position im Vektor repräsentiert wird. Jede Position im Vektor steht für eine mögliche Kategorie des Merkmals. Wenn ein bestimmtes Element eines Merkmals vorhanden ist, wird der entsprechende Wert in der Vektorrepräsentation auf 1 gesetzt. Andernfalls wird der Wert auf 0 gesetzt.

In [ ]:
text_vectorization = TextVectorization(
 max_tokens=20000,
ngrams=3,
 output_mode="multi_hot",
)

In [ ]:
text_vectorization.adapt(train_data) 
text_vectorization.adapt(test_data) 

multi_hot_train_data = text_vectorization(train_data)
multi_hot_test_data = text_vectorization(test_data)

### Hot Encoding der Lables
Multi Hot Encoding nutzen wir auch für die Labels.

In [ ]:
train_label_one_hot = tf.one_hot(train_label, 5)
train_label_one_hot.shape

## Erstellung eines entsprechenden künstlichen neuronalen Models sowie geeigente Benchmarks
Damit wir hier ein Softmax output Layer nutzen können um in 5 Klassen die Daten Klassifizieren können.
Als Besonderheit nutzen wir noch ein Drop out Layer. 
Ein Dropout-Layer ist eine spezielle Art von Schicht in einem neuronalen Netzwerk, die dazu dient, Overfitting (Überanpassung) zu reduzieren. Overfitting tritt auf, wenn ein neuronales Netzwerk sehr gut auf die Trainingsdaten passt, aber eine schlechte Leistung auf neuen, unbekannten Daten zeigt.

Der Dropout-Layer funktioniert, indem er während des Trainings zufällig eine bestimmte Anzahl von Ausgabefunktionen (Neuronen) "auslässt" oder "fallen lässt". Das bedeutet, dass diese Neuronen während eines bestimmten Trainingsdurchlaufs keine Informationen an die nachfolgende Schicht weitergeben.

In [14]:
def get_model(max_tokens=20000, hidden_dim=64):
 inputs = keras.Input(shape=(max_tokens,))
 x = layers.Dense(hidden_dim, activation="relu")(inputs)
 x = layers.Dense(32, activation="relu")(x)
 x = layers.Dropout(0.5)(x)
 x = layers.Dense(16, activation="relu")(x)
 outputs = layers.Dense(5, activation="softmax")(x)
 model = keras.Model(inputs, outputs)
 model.compile(optimizer="rmsprop",
 loss="binary_crossentropy",
 metrics=["accuracy"], 
 )
 return model

model = get_model()
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 64)                1280064   
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 5)                 85        
                                                                 
Total params: 1,282,757
Trainable params: 1,282,757
Non-tra

## Durchführung von In-sample und Out-of-sample Prognosen sowie Bestimmungen sowie Bestimmung der Prognossegüter anhand geeigneter Metriken 

In [15]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]
history = model.fit(multi_hot_train_data, train_label_one_hot, validation_split=0.4, epochs=10, callbacks= callbacks)

Epoch 1/10
501/501 [==============================] - 9s 16ms/step - loss: 0.3422 - accuracy: 0.3673 - val_loss: 0.2560 - val_accuracy: 0.4566
Epoch 2/10
501/501 [==============================] - 8s 17ms/step - loss: 0.2406 - accuracy: 0.4695 - val_loss: 0.2426 - val_accuracy: 0.4680
Epoch 3/10
501/501 [==============================] - 8s 15ms/step - loss: 0.2109 - accuracy: 0.5103 - val_loss: 0.2423 - val_accuracy: 0.4734
Epoch 4/10
501/501 [==============================] - 7s 14ms/step - loss: 0.1814 - accuracy: 0.5440 - val_loss: 0.2613 - val_accuracy: 0.4784
Epoch 5/10
501/501 [==============================] - 8s 15ms/step - loss: 0.1578 - accuracy: 0.5711 - val_loss: 0.2781 - val_accuracy: 0.4737
Epoch 6/10
501/501 [==============================] - 9s 18ms/step - loss: 0.1404 - accuracy: 0.5877 - val_loss: 0.2906 - val_accuracy: 0.4709


In [ ]:
from utils.plot_utils import plot_history_metrics
plot_history_metrics(history, ['loss', 'accuracy'])

In [17]:
test_label_one_hot = tf.one_hot(test_label, 5)
model.evaluate(multi_hot_test_data,test_label_one_hot)

209/209 [==============================] - 1s 3ms/step - loss: 0.2472 - accuracy: 0.4642


[0.24722281098365784, 0.4642215669155121]

## Fazit

Wir kommen auf Out-of Samlple accuracy von 45%. Als Ziel haben wir uns 20 % Prozent gesetzt, weil wir 5 verschiedene Lables haben und alles drunter nur Raten wäre. Somit hat unsere Analyse das Ziel erreicht.